<a href="https://colab.research.google.com/github/jhfd16/santander/blob/main/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [9]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [10]:
import pandas as pd

df = pd.read_csv('devweek.csv')
user_ids = df['User ID'].tolist()
print(user_ids)

[5566, 5568, 5569]


In [11]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 5566,
    "name": "Jo\u00e3o",
    "account": {
      "id": 5898,
      "number": "777777777-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 5406,
      "number": "**** **** ***1 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5568,
    "name": "Henrique",
    "account": {
      "id": 5900,
      "number": "777777777-2",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 5408,
      "number": "**** **** ***2 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5569,
    "name": "Dias",
    "account": {
      "id": 5901,
      "number": "777777777-3",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 5409,
      "number": "**** **** ***3 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [38]:
import random

lista_frases = [', invista mais!', ', está rico!' , "investimentos são a chave para o futuro financeiro. Cresça seu dinheiro, não apenas o guarde!"]
for user in users:
  print(user['name'], lista_frases[random.randint(0,2)])

João , invista mais!
Henrique , está rico!
Dias investimentos são a chave para o futuro financeiro. Cresça seu dinheiro, não apenas o guarde!


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [39]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User João updated? True!
User Henrique updated? True!
User Dias updated? True!
